<IMG SRC="https://avatars2.githubusercontent.com/u/31697400?s=400&u=a5a6fc31ec93c07853dd53835936fd90c44f7483&v=4" WIDTH=125 ALIGN="right">



# Surface water example
   
*D.A. Brakenhoff, Artesia, 2020*

This example notebook shows some how to add surface water defined in a shapefile to a MODFLOW model using the `nlmod` package.
    
    
### Contents<a id='top'></a>

1. [Load data](#1)
2. [Build model](#2)
3. [Add surface water](#3)
    1. [Intersect surface water shape with grid](#3.1)
    2. [Aggregate parameters per model cell](#3.2)
    3. [Build stress period data](#3.3)
    4. [Create RIV package](#3.4)
4. [Write + run model](#4)
5. [Visualize results](#5)

### TODO:

- Transient boundary conditions (i.e. functionality to use strings for parameters and write timeseries)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import flopy
import logging

import nlmod

In [ ]:
print(f'nlmod version: {nlmod.__version__}')

# toon informatie bij het aanroepen van functies
logging.basicConfig(level=logging.INFO)

## [1. Load data](#top)<a id='1'></a>
Load shapefile with surface water features. 

In [ ]:
sfw = gpd.read_file("../data/shapes/schnhvn_opp_water.shp")
# vervang peilvak_id met None door 'None'
sfw.loc[sfw.peilvak_id.isna(), 'peilvak_id'] = 'None'

Take a look at the first few rows. For adding surface water features to a MODFLOW model the following attributes must be present:

- **stage**: the water level (in m NAP)
- **botm**: the bottom elevation (in m NAP)
- **c0**: the bottom resistance (in days)

The `stage` and the `botm` columns are present in our dataset. The bottom resistance `c0` is rarely known, and is usually estimated when building the model. We will add our estimate later on.

*__Note__: the NaN's in the dataset indicate that not all parameters are known for each feature. This is not necessarily a problem but this will mean some features will not be converted to model input.*

Plot the surface water features using the column `peilvak_id` to color the features. 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
ax.set_aspect("equal", adjustable="box")
sfw.plot(ax=ax, column="peilvak_id")
ax.grid(b=True)
ax.set_xlabel("X (m RD)")
ax.set_ylabel("Y (m RD)")
plt.yticks(rotation=90, va="center")
fig.tight_layout()

Now use `stage` as the column to color the data. Note the missing features caused by the fact that the stage is undefined (NaN).

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
ax.set_aspect("equal", adjustable="box")
sfw.plot(ax=ax, column="stage", legend=True)
ax.grid(b=True)
ax.set_xlabel("X (m RD)")
ax.set_ylabel("Y (m RD)")
plt.yticks(rotation=90, va="center")
fig.tight_layout()

## [2. Build model](#top)<a id='2'></a>

The next step is to define a model grid and build a model (i.e. create a discretization and define flow parameters). First we define the extent of our model and subsequently input that information into the convenient methods in `nlmod` to download all the relevant data and create a Modflow6 model. 

In [ ]:
extent = [115900, 121000, 436600, 442000]  # Schoonhoven

Build the model. We're keeping the model as simple as possible.

In [ ]:
use_cache = True
model_name = "model2"
model_ws = "./model2"

delr = delc = 50.0
start_time = "2021-01-01"

In [ ]:
# create model time dataset
model_ds = nlmod.mdims.get_empty_model_ds(model_name, model_ws)
model_ds = nlmod.mdims.set_model_ds_time(model_ds,
                                         start_time=start_time,
                                         steady_state=True)


extent, nrow, ncol = nlmod.read.regis.fit_extent_to_regis(extent, delr, delc)

# layer model
layer_model = nlmod.read.regis.get_combined_layer_models(extent,
                                                         delr, delc,
                                                         use_regis=True,
                                                         use_geotop=False,
                                                         cachedir=model_ds.cachedir,
                                                         cachename='combined_layer_ds.nc')

# create modflow packages
sim, gwf = nlmod.mfpackages.sim_tdis_gwf_ims_from_model_ds(model_ds)

In [ ]:
# update model_ds from layer model
model_ds = nlmod.mdims.update_model_ds_from_ml_layer_ds(model_ds,
                                                        layer_model,
                                                        keep_vars=['x', 'y'],
                                                        add_northsea=False,
                                                        cachedir=model_ds.cachedir)

# Create discretization
dis = nlmod.mfpackages.dis_from_model_ds(model_ds, gwf)

# create node property flow
npf = nlmod.mfpackages.npf_from_model_ds(model_ds, gwf)

# Create the initial conditions package
ic = nlmod.mfpackages.ic_from_model_ds(model_ds, gwf, starting_head=1.0)

# Create the output control package
oc = nlmod.mfpackages.oc_from_model_ds(model_ds, gwf)

## [3. Add surface water](#top)<a id='3'></a>

Now that we have a discretization (a grid, and layer tops and bottoms) we can start processing our surface water shapefile to add surface water features to our model. The method to add surface water starting from a shapefile is divided into the following steps:

1. Intersect surface water shape with grid. This steps intersects every feature with the grid so we can determine the surface water features in each cell.
2. Aggregate parameters per grid cell. Each feature within a cell has its own parameters. For MODFLOW it is often desirable to have one representative set of parameters per cell. These representative parameters are calculated in this step.
3. Build stress period data. The results from the previous step are converted to stress period data (generally a list of cellids and representative parameters: `[(cellid), parameters]`) which is used by MODFLOW and flopy to define boundary conditions.
4. Create the Modflow6 package

The steps are illustrated below.

### [Intersect surface water shape with grid](#top)<a id='3.1'></a>

The first step is to intersect the surface water shapefile with the grid.

In [ ]:
sfw_grid = nlmod.mdims.gdf2grid(sfw, gwf, method='vertex')

Plot the result and the model grid and color using `cellid`. It's perhaps a bit hard to see but each feature is cut by the gridlines. 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
ax.set_aspect("equal", adjustable="box")
sfw_grid.plot(ax=ax, column="cellid")
gwf.modelgrid.plot(ax=ax, linewidth=0.5, color="k")
xmin, xmax, ymin, ymax = extent
offset = 100
ax.set_xlim(xmin-offset, xmax+offset)
ax.set_ylim(ymin-offset, ymax+offset);
fig.savefig(os.path.join(model_ds.figdir,'surface_water_Schoonhoven.png'))

### [Aggregate parameters per model cell](#top)<a id='3.2'></a>

The next step is to aggregate the parameters for all the features in one grid cell to obtain one representative set of parameters. First, let's take a look at a grid cell containing multiple features.

In [ ]:
cid = (107, 6)  # for 50 x 50 m grid
# cid = (5, 45)  # for 100 x 100 m grid
mask = sfw_grid.cellid == cid
sfw_grid.loc[mask]

We can also plot the features within that grid cell.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
sfw_grid.loc[mask].plot(column="unique_id", legend=True, ax=ax,
                        legend_kwds={"loc": "upper left"})
xlim = ax.get_xlim()
ylim = ax.get_ylim()
gwf.modelgrid.plot(ax=ax)
ax.set_xlim(xlim[0], xlim[0]+model_ds.delr*1.1)
ax.set_ylim(ylim)
ax.set_title(f"Surface water shapes in cell: {cid}");
fig.savefig(os.path.join(model_ds.figdir,'surface_water_detail.png'))

Now we want to aggregate the features in each cell to obtain a representative set of parameters (`stage`, `conductance`, `bottom elevation`) to use in the model. There are several aggregation methods. Note that the names of the methods are not representative of the aggregation applied to each parameter. For a full description see the following list:

- `'area_weighted'`
  - **stage**: area-weighted average of stage in cell
  - **cond**: conductance is equal to area of surface water divided by bottom resistance
  - **elev**: the lowest bottom elevation is representative for the cell
- `'max_area'`
  - **stage**: stage is determined by the largest surface water feature in a cell
  - **cond**: conductance is equal to area of all surface water features divided by bottom resistance
  - **elev**: the lowest bottom elevation is representative for the cell
- `'de_lange'`
  - **stage**: area-weighted average of stage in cell
  - **cond**: conductance is calculated using the formulas derived by De Lange (1999).
  - **elev**: the lowest bottom elevation is representative for the cell
  
Let's try using `area_weighted`. This means the stage is the area-weighted average of all the surface water features in a cell. The conductance is calculated by dividing the total area of surface water in a cell by the bottom resistance (`c0`). The representative bottom elevation is the lowest elevation present in the cell.

In [ ]:
try:
    nlmod.mfpackages.surface_water.aggregate_surface_water(
        sfw_grid, "area_weighted")
except ValueError as e:
    print(e)

The function checks whether the requisite columns are defined in the DataFrame. We need to add a column containing the bottom resistance `c0`. Often a value of 1 day is used as an initial estimate.

In [ ]:
sfw_grid["c0"] = 1.0  # days

Now aggregate the features.

In [ ]:
celldata = nlmod.mfpackages.surface_water.aggregate_surface_water(sfw_grid, "area_weighted")

Let's take a look at the result. We now have a DataFrame with cell-id as the index and the three parameters we need for each cell `stage`, `cond` and `rbot`. The area is also given, but is not needed for the groundwater model. 

In [ ]:
celldata.head(10)

### [Build stress period data](#top)<a id='3.3'></a>

The next step is to take our cell-data and build convert it to 'stress period data' for MODFLOW. This is a data format that defines the parameters in each cell in the following format:

```
[[(cellid1), param1a, param1b, param1c],
 [(cellid2), param2a, param2b, param2c],
 ...]
```

The required parameters are defined by the MODFLOW-package used:

- **RIV**: for the river package `(stage, cond, rbot)`
- **DRN**: for the drain package `(stage, cond)`
- **GHB**: for the general-head-boundary package `(stage, cond)`

We're selecting the RIV package. We don't have a bottom (rbot) for each reach in celldata. Therefore we remove the reaches where rbot is nan (not a number).

In [ ]:
new_celldata = celldata.loc[~celldata.rbot.isna()]
print(f'removed {len(celldata)-len(new_celldata)} reaches because rbot is nan')

In [ ]:
riv_spd = nlmod.mfpackages.surface_water.build_spd(new_celldata, "RIV", model_ds)

Take a look at the stress period data for the river package:

In [ ]:
riv_spd[:10]

### [Create RIV package](#top)<a id='3.4'></a>

The final step is to create the river package using flopy.

In [ ]:
riv = flopy.mf6.ModflowGwfriv(gwf, stress_period_data=riv_spd)

Plot the river boundary condition to see where rivers were added in the model

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 8), constrained_layout=True)
mv = flopy.plot.PlotMapView(model=gwf, ax=ax, layer=0)
mv.plot_bc("RIV");

## [4. Write + run model](#top)<a id='4'></a>

Now write the model simulation to disk, and run the simulation.

In [ ]:
nlmod.util.write_and_run_model(gwf, model_ds, write_model_ds=True, nb_path='02_surface_water.ipynb')

## [5. Visualize results](#top)<a id='5'></a>

To see whether our surface water was correctly added to the model, let's visualize the results. We'll load the calculated heads, and plot them.

In [ ]:
hds_obj = flopy.utils.HeadFile(os.path.join(
    model_ds.model_ws, model_ds.model_name) + ".hds")

Load the data, and set NODATA (often values of +1e30) to `np.nan`.

In [ ]:
h = hds_obj.get_alldata()
h[h > 1e20] = np.nan  # set NODATA to NaN

Plot the heads in a specific model layer

In [ ]:
ilay = 0
fig, ax = plt.subplots(1, 1, figsize=(10, 8), constrained_layout=True)
mv = flopy.plot.PlotMapView(model=gwf, ax=ax, layer=ilay)
qm = mv.plot_array(h[-1], cmap="RdBu")  # last timestep
mv.plot_ibound()  # plot inactive cells in red
fig.colorbar(qm, shrink=1.0)
ax.set_title(f"Heads top-view, layer {ilay}");
fig.savefig(os.path.join(model_ds.figdir,f'heads_layer{ilay}.png'))

In cross-section

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 3), constrained_layout=True)
xs = flopy.plot.PlotCrossSection(
    model=gwf, ax=ax, line={"row": gwf.modelgrid.nrow // 2})
qm = xs.plot_array(h[-1], cmap="RdBu")  # last timestep
xs.plot_ibound()  # plot inactive cells in red
fig.colorbar(qm, shrink=1.0)
row = gwf.modelgrid.nrow // 2
ax.set_title(f"Cross-section along row {row}");
fig.savefig(os.path.join(model_ds.figdir,f'heads_cross_section_along_row{row}.png'))